In [ ]:
import numpy as np
import pandas as pd

import requests
import zipfile
import os
import sys

In [ ]:
# Define the paths
dataset_url = "https://github.com/skoltech-nlp/detox/releases/download/emnlp2021/filtered_paranmt.zip"
zip_file_path = "dataset.zip"
csv_file_path, tsv_file_path = "dataset.csv", "filtered.tsv"

# Download the ZIP file
response = requests.get(dataset_url)
if response.status_code == 200:
    with open(zip_file_path, 'wb') as file:
        file.write(response.content)
else:
    print("Attempt failed")
    exit()

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(".")

dataset = pd.read_csv("filtered.tsv", delimiter='\t')
dataset.to_csv(csv_file_path, index=False)

# ZIP cleaning up
os.remove(zip_file_path)
os.remove(tsv_file_path)

In [ ]:
dataset.sample(5)

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
419610,419610,It'll be the fall of Saigon if I'm late.,"if I'm late, she'll be hit.",0.714844,0.317073,0.000104,0.838105
167521,167521,"Cruisin', gunning the engine so loud, nerds wo...",just drive up and dial the engine so that the ...,0.671165,0.039474,0.058586,0.952140
252613,252613,"I am an adult, damn it!""","I'm an adult! """,0.765863,0.360000,0.994764,0.000303
312041,312041,"I killed then, but I did not like it.","I killed them, but I didn't like it.",0.920389,0.026316,0.020455,0.963189
395031,395031,He would have me dead.,he would have wanted me.,0.701146,0.080000,0.991558,0.000057


In [ ]:
repo_dir = "nlp-Text-De-toxification"

if os.path.exists(repo_dir):
    print(f"{repo_dir} already exists. Removing it...\n")
    !rm -r {repo_dir}

# Clone the repository from GitHub
!git clone https://github.com/Goshmar/nlp-Text-De-toxification

Cloning into 'nlp-Text-De-toxification'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 78 (delta 29), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (78/78), 2.00 MiB | 4.60 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [ ]:
! pip install -r nlp-Text-De-toxification/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.4/677.4 kB 37.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.8/400.8 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788

In [ ]:
def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

# adding the path to the condebert code root
add_sys_path('nlp-Text-De-toxification/src')

/content/nlp-Text-De-toxification/src


In [ ]:
from importlib import reload

sys.path.append('/content/nlp-Text-De-toxification/src/models')

import condbert

from condbert import CondBertRewriter


In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import numpy as np
import pickle
from tqdm.auto import tqdm, trange

In [ ]:
device = torch.device('cpu')  # please change it to e.g. 'cuda:0' if you want to use a GPU

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
# Downloading vocab for CondBert model
def load_words_from_file(file_path):
    with open(file_path, "r") as file:
        words = [line.strip() for line in file.readlines()]
    return words

vocab_root = "nlp-Text-De-toxification/data/vocab_1/"
negative_words = load_words_from_file(vocab_root + "negative-words.txt")
toxic_words = load_words_from_file(vocab_root + "toxic_words.txt")
positive_words = load_words_from_file(vocab_root + "positive-words.txt")

In [ ]:
import pickle
with open(vocab_root + 'word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [ ]:
token_toxicities = []
with open(vocab_root + 'token_toxicities.txt', 'r') as f:
    for line in f.readlines():
        token_toxicities.append(float(line))
token_toxicities = np.array(token_toxicities)
token_toxicities = np.maximum(0, np.log(1/(1/token_toxicities-1)))   # log odds ratio
# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_toxicities[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_toxicities[tokenizer.encode(tok)][1] = 0

In [ ]:
reload(condbert)
from condbert import CondBertRewriter

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
)

In [ ]:
print(editor.translate('You are an idiot!'))

You are an idiot!
you are an the !


In [ ]:
editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
    predictor=None,
)

In [ ]:
import masked_token_predictor_bert
reload(masked_token_predictor_bert)

from masked_token_predictor_bert import MaskedTokenPredictorBert

In [ ]:
predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0)
editor.predictor = predictor

def adjust_logits(logits, label):
    return logits - editor.token_toxicities * 3

predictor.logits_postprocessor = adjust_logits

print(editor.replacement_loop('You are an idiot!', verbose=False))

you are an old man !


In [ ]:
import choosers
reload(choosers)
from choosers import EmbeddingSimilarityChooser

In [ ]:
predictor = MaskedTokenPredictorBert(
    model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0,
    confuse_bert_args=True, # this argument deteriorates quality but is used for backward compatibility
)
editor.predictor = predictor

def adjust_logits(logits, label=0):
    return logits - editor.token_toxicities * 10

predictor.logits_postprocessor = adjust_logits

cho = EmbeddingSimilarityChooser(sim_coef=100, tokenizer=tokenizer)

2023-10-21 19:06:03,200 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpel7taytu


100%|██████████| 160000128/160000128 [00:09<00:00, 17251325.61B/s]

2023-10-21 19:06:13,121 copying /tmp/tmpel7taytu to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2023-10-21 19:06:13,861 removing temp file /tmp/tmpel7taytu
2023-10-21 19:06:14,682 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmp7etnzslu


100%|██████████| 21494764/21494764 [00:01<00:00, 13851046.41B/s]

2023-10-21 19:06:16,890 copying /tmp/tmp7etnzslu to cache at /root/.flair/embeddings/glove.gensim
2023-10-21 19:06:16,919 removing temp file /tmp/tmp7etnzslu


In [ ]:
for example in tqdm(dataset.sample(5)['reference']):
    print("------")
    print(example)
    print("-->", editor.replacement_loop(example, verbose=False, chooser=cho, n_top=10, n_tokens=(1,2,3), n_units=1))
    print("------\n\n")

  0%|          | 0/5 [00:00<?, ?it/s]

------
Shut up, Nick. You'll hurt Coco Chanel's feelings.
--> . up , " " . . " . . ' ll . coco chanel ' s feelings .
------


------
If what I felt had been an actual wound, I'd have been bleeding to death.
--> if what i felt had been an actual i , i ' d have been i to " . . .
------


------
And that we became friends in the process... ...and learned some shit about life and stuff.
--> and that we became friends in the process . . . . . . and learned some " " . about " . . and stuff .
------


------
You smell like a hobo.
--> i i like a " " " .
------


------
I'd kill my first born for a caramel macchiato.
--> i ' d " . . " . . first . " . for a caramel macchiato .
------




In [ ]:
VOCAB_DIRNAME = 'vocabularies'

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.to(device);

In [ ]:
tox_corpus_path = '/content/nlp-Text-De-toxification/data/train_toxic.txt'
norm_corpus_path = '/content/nlp-Text-De-toxification/data/train_normal.txt'

if not os.path.exists(VOCAB_DIRNAME):
    os.makedirs(VOCAB_DIRNAME)

In [ ]:
import os
import argparse
import numpy as np
from tqdm import tqdm
from nltk import ngrams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



class NgramSalienceCalculator():
    def __init__(self, tox_corpus, norm_corpus, use_ngrams=False):
        ngrams = (1, 3) if use_ngrams else (1, 1)
        self.vectorizer = CountVectorizer(ngram_range=ngrams)

        tox_count_matrix = self.vectorizer.fit_transform(tox_corpus)
        self.tox_vocab = self.vectorizer.vocabulary_
        self.tox_counts = np.sum(tox_count_matrix, axis=0)

        norm_count_matrix = self.vectorizer.fit_transform(norm_corpus)
        self.norm_vocab = self.vectorizer.vocabulary_
        self.norm_counts = np.sum(norm_count_matrix, axis=0)

    def salience(self, feature, attribute='tox', lmbda=0.5):
        assert attribute in ['tox', 'norm']
        if feature not in self.tox_vocab:
            tox_count = 0.0
        else:
            tox_count = self.tox_counts[0, self.tox_vocab[feature]]

        if feature not in self.norm_vocab:
            norm_count = 0.0
        else:
            norm_count = self.norm_counts[0, self.norm_vocab[feature]]

        if attribute == 'tox':
            return (tox_count + lmbda) / (norm_count + lmbda)
        else:
            return (norm_count + lmbda) / (tox_count + lmbda)


In [ ]:
from collections import Counter
c = Counter()

for fn in [tox_corpus_path, norm_corpus_path]:
    with open(fn, 'r') as corpus:
        for line in corpus.readlines():
            for tok in line.strip().split():
                c[tok] += 1

print(len(c))

88645


In [ ]:
vocab = {w for w, _ in c.most_common() if _ > 0}  # if we took words with > 1 occurences, vocabulary would be x2 smaller, but we'll survive this size
print(len(vocab))

88645


In [ ]:
with open(tox_corpus_path, 'r') as tox_corpus, open(norm_corpus_path, 'r') as norm_corpus:
    corpus_tox = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in tox_corpus.readlines()]
    corpus_norm = [' '.join([w if w in vocab else '<unk>' for w in line.strip().split()]) for line in norm_corpus.readlines()]

In [ ]:
neg_out_name = '/content/nlp-Text-De-toxification/data/vocab_1/negative-words.txt'
pos_out_name = '/content/nlp-Text-De-toxification/data/vocab_1/positive-words.txt'

In [ ]:
threshold = 4
sc = NgramSalienceCalculator(corpus_tox, corpus_norm, False)
seen_grams = set()

with open(neg_out_name, 'w') as neg_out, open(pos_out_name, 'w') as pos_out:
    for gram in set(sc.tox_vocab.keys()).union(set(sc.norm_vocab.keys())):
        if gram not in seen_grams:
            seen_grams.add(gram)
            toxic_salience = sc.salience(gram, attribute='tox')
            polite_salience = sc.salience(gram, attribute='norm')
            if toxic_salience > threshold:
                neg_out.writelines(f'{gram}\n')
            elif polite_salience > threshold:
                pos_out.writelines(f'{gram}\n')

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

In [ ]:
X_train = corpus_tox + corpus_norm
y_train = [1] * len(corpus_tox) + [0] * len(corpus_norm)
pipe.fit(X_train, y_train);

In [ ]:
coefs = pipe[1].coef_[0]
coefs.shape

(88519,)

In [ ]:
word2coef = {w: coefs[idx] for w, idx in pipe[0].vocabulary_.items()}

In [ ]:
import pickle
with open('/content/nlp-Text-De-toxification/data/vocab_1/word2coef.pkl', 'wb') as f:
    pickle.dump(word2coef, f)

In [ ]:
from collections import defaultdict
toxic_counter = defaultdict(lambda: 1)
nontoxic_counter = defaultdict(lambda: 1)

for text in tqdm(corpus_tox):
    for token in tokenizer.encode(text):
        toxic_counter[token] += 1
for text in tqdm(corpus_norm):
    for token in tokenizer.encode(text):
        nontoxic_counter[token] += 1

100%|██████████| 135390/135390 [01:37<00:00, 1387.27it/s]


In [ ]:
token_toxicities = [toxic_counter[i] / (nontoxic_counter[i] + toxic_counter[i]) for i in range(len(tokenizer.vocab))]

In [ ]:
with open('/content/nlp-Text-De-toxification/data/vocab_1/token_toxicities.txt', 'w') as f:
    for t in token_toxicities:
        f.write(str(t))
        f.write('\n')

In [ ]:
with open('/content/nlp-Text-De-toxification/data/vocab_1/negative-words.txt', "r") as f:
    s = f.readlines()
negative_words = list(map(lambda x: x[:-1], s))

with open('/content/nlp-Text-De-toxification/data/vocab_1/positive-words.txt', "r") as f:
    s = f.readlines()
positive_words = list(map(lambda x: x[:-1], s))

FileNotFoundError: ignored

In [ ]:
import pickle
with open('/content/nlp-Text-De-toxification/data/vocab_1/word2coef.pkl', 'rb') as f:
    word2coef = pickle.load(f)

In [ ]:
token_toxicities = []
with open('/content/nlp-Text-De-toxification/data/vocab_1/token_toxicities.txt', 'r') as f:
    for line in f.readlines():
        token_toxicities.append(float(line))
token_toxicities = np.array(token_toxicities)
token_toxicities = np.maximum(0, np.log(1/(1/token_toxicities-1)))   # log odds ratio

# discourage meaningless tokens
for tok in ['.', ',', '-']:
    token_toxicities[tokenizer.encode(tok)][1] = 3

for tok in ['you']:
    token_toxicities[tokenizer.encode(tok)][1] = 0

In [ ]:
def adjust_logits(logits, label=0):
    return logits - token_toxicities * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(model, tokenizer, max_len=250, device=device, label=0, contrast_penalty=0.0, logits_postprocessor=adjust_logits)

editor = CondBertRewriter(
    model=model,
    tokenizer=tokenizer,
    device=device,
    neg_words=negative_words,
    pos_words=positive_words,
    word2coef=word2coef,
    token_toxicities=token_toxicities,
    predictor=predictor,
)

In [ ]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

In [ ]:
print(editor.translate('You are idiot!', prnt=False))

you are mistake !


In [ ]:
print(editor.replacement_loop('You are stupid!', verbose=False, chooser=chooser, n_tokens=(1, 2, 3), n_top=10))

you are very beautiful !


In [ ]:
for example in dataset.sample(5)['reference']:
    print("------")
    print(example)
    print("-->", editor.replacement_loop(example, verbose=False, chooser=chooser, n_tokens=(1, 2, 3), n_top=10))
    print("------\n\n")

------
You still upset that I killed your buddy?
--> you still upset that i went after my buddy ?
------


------
Boy, it's... it's funny about f-finding this fucking plane, wasn't it?
--> boy , it ' s . . . it ' s funny about f - finding this on the wrong plane , wasn ' t it ?
------


------
You got me started with cooking, mrs.
--> " they got i started with cooking , mrs .
------


------
Oh, I'm so sick of this shit!
--> oh , i ' m so not sure any of this stuff !
------


------
You will never drink my piss.
--> you will never drink my sway .
------


